# Численное интегрирование



In [1]:
%load_ext Cython

## Метод прямоугольников

Сумма Римана: $$\int_a^b f(x) dx \approx \sum_{i=1}^n f(x^*_i) \Delta x_i,\ \Delta x_i = x_i - x_{i-1}$$

### Метод левых прямоугольников (левая сумма)

В качестве $x^*_i$ выбираются левые границы отрезков: $x^*_i = x_{i-1}$.

In [2]:
%%cython
def left_riemann_sum(f, float a, float b, int n):
  def x(int i): return a + i*((b - a) / n)

  return sum(f(x(i-1)) * (x(i) - x(i-1)) for i in range(1, n + 1))

### Метод правых прямоугольников (правая сумма)

В качестве $x^*_i$ выбираются правые границы отрезков: $x^*_i = x_i$.

In [3]:
%%cython
def right_riemann_sum(f, float a, float b, int n):
  def x(int i): return a + i*((b - a) / n)

  return sum(f(x(i)) * (x(i) - x(i-1)) for i in range(1, n + 1))

### Метод средних прямоугольников (средняя сумма)

В качестве $x^*_i$ выбираются середины отрезков: $x^*_i = \frac{1}{2}(x_{i-1} + x_i)$.

In [4]:
%%cython
def midpoint_riemann_sum(f, float a, float b, int n):
  def x(int i): return a + i*((b - a) / n)

  return sum(f((x(i-1) + x(i)) / 2) * (x(i) - x(i-1)) for i in range(1, n + 1))

## Метод трапеций

$$\int_a^b f(x) dx \approx \sum_{i=1}^n \frac{f(x_{i-1}) + f(x_i)}{2}\Delta x_i,\ \Delta x_i = x_i - x_{i-1}$$

In [5]:
%%cython
def trapezoidal_rule(f, float a, float b, int n):
  def x(int i): return a + i*((b - a) / n)
  
  return sum((f(x(i-1)) + f(x(i))) / 2 * (x(i) - x(i-1)) for i in range(1, n + 1))

## Метод Симпсона

$$\int_a^b f(x) dx \approx \frac{1}{3} \frac{(b - a)}{n} \bigg[
f(x_0) + 4\sum_{i=1,3...}^{n-1} f(x_i) + 2 \sum_{i=2,4,...}^{n-2} f(x_i) + f(x_n))
\bigg]$$

In [6]:
%%cython
def simpsons_rule(f, float a, float b, int n):
  def x(int i): return a + i*((b - a) / n)

  return (1 / 3) * ((b - a) / n) * (
    f(x(0)) + 4*sum(f(x(i)) for i in range(1, n, 2)) + 2*sum(f(x(i)) for i in range(2, n, 2)) + f(x(n)))

## Точность расчетов

In [7]:
def iterate_upto_precision(method, f, a, b, precision=10**-6):
  n = 4
  res_n, res2n = 0, 0
  while True:
    res_n = method(f, a, b, n)
    res_2n = method(f, a, b, 2*n)
    if abs(res_n - res_2n) > precision:
      n *= 2
    else:
      return n, res_2n

## Пользовательский интерфейс

In [8]:
from sympy.parsing.sympy_parser import parse_expr, standard_transformations,\
  implicit_multiplication_application, implicit_application, convert_xor
from sympy.utilities.autowrap import autowrap

def parse_function(raw_expr):
  transformations = (standard_transformations +
    (implicit_multiplication_application, implicit_application, convert_xor))
  f = parse_expr(raw_expr, transformations=transformations)
  assert list(map(str, f.free_symbols)) == ['x'], \
    'Функция может содержать только одну свободную переменную, x'
  return str(f), autowrap(f)

In [9]:
from IPython.display import display
import ipywidgets as widgets

def eval_function(f_string):
  label, f = parse_function(f_string)
  print(f'Введенная функция: y = {label}')
  
  def approx_integrate(a, b, precision, method):
    method_f = (left_riemann_sum if method == 'левых прямоугольников' else
                right_riemann_sum if method == 'правых прямоугольников' else
                midpoint_riemann_sum if method == 'средних прямоугольников' else
                trapezoidal_rule if method == 'трапеций' else
                simpsons_rule)
    print(f'Интегрирование методом {method}...')
    n, res = iterate_upto_precision(method_f, f, a, b, precision)
    print(f'Результат: {res}, число разбиений интервала n = {n}')
  
  a = widgets.FloatText(description='a', value=0.0, step=0.1)
  b = widgets.FloatText(description='b', value=0.0, step=0.1)
  precision = widgets.FloatText(description='Δ', value=0.01, step=0.000001)
  method = widgets.RadioButtons(description='Метод:',
    options=['левых прямоугольников', 'правых прямоугольников', 'средних прямоугольников', 'трапеций', 'Симпсона']) 
  widgets.interact(approx_integrate, a=a, b=b, precision=precision, method=method)

print('Введите функцию с одной переменной для интегрирования:')
print('Примеры функций: x * sin 2x, x^2 + 3x + 12, (x+3) / 2^x')
inp = widgets.Text(placeholder='f(x)')
inp_button = widgets.Button(description='Ввод')
inp_button.on_click(lambda _: eval_function(inp.value))
display(widgets.HBox([inp, inp_button]))

Введите функцию с одной переменной для интегрирования:
Примеры функций: x * sin 2x, x^2 + 3x + 12, (x+3) / 2^x


Введенная функция: y = x*sin(2*x)


interactive(children=(FloatText(value=0.0, description='a', step=0.1), FloatText(value=0.0, description='b', s…